https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html

In [24]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

1.0.1
1.18.1


# Pandas MultiIndex
Multiple levels of index

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [4]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [5]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

# MultiIndex as extra dimension
Multiply indexed Series -> Data Frame

In [7]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [8]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [9]:
# Add column to DataFrame
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [10]:
# ufuncs | Oerations: fraction of people under 18
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


# MultiIndex Creation

In [11]:
# pass a list of arrays to the constructor
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.960423  0.201764
  2  0.049815  0.398889
b 1  0.611331  0.743381
  2  0.216472  0.946290

In [12]:
# pass a dictionary with appropriate tuples as keys
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

# Explicit MultiIndex constructors

In [13]:
#From Arrays
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [15]:
# From Tuples
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [16]:
#From Cartesian Product
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [25]:
# By Levels
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

TypeError: __new__() got an unexpected keyword argument 'labels'

# MultiIndex level names

In [26]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

# MultiIndex for Columns
Columns can have multiple levels

In [36]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 2) # 4 rows | 6 clumns | decinals
#data[:, ::2] *= 10
#data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      1.82  0.72 -2.83 -0.14 -0.40 -0.48
     2      0.46  0.56  0.89 -0.27  1.28 -0.93
2014 1     -0.48 -0.58  0.54 -0.25  1.32 -1.38
     2     -1.67  0.79 -1.23  0.79 -0.17  0.75

In [37]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1     -2.83 -0.14
     2      0.89 -0.27
2014 1      0.54 -0.25
     2     -1.23  0.79

# Indexing and Slicing a MultiIndex

In [39]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [38]:
# indexing by multiple terms
pop['California', 2000]

33871648

In [40]:
# indexing just one level -> returns a Series
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [41]:
#partial slicing
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [42]:
#partial indexing
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [43]:
#selection based on Boolean mask
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [44]:
#fancy indexing
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

# Multiply indexed DataFrames

In [45]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      1.82  0.72 -2.83 -0.14 -0.40 -0.48
     2      0.46  0.56  0.89 -0.27  1.28 -0.93
2014 1     -0.48 -0.58  0.54 -0.25  1.32 -1.38
     2     -1.67  0.79 -1.23  0.79 -0.17  0.75

In [46]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      1.82  0.72
     2      0.46  0.56

In [47]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        1.82
      2        0.46
2014  1       -0.48
      2       -1.67
Name: (Bob, HR), dtype: float64

In [49]:
# IndexSlice object
idx = pd.IndexSlice
health_data.loc[idx[:, 2], idx[:, 'HR']]
# visits = 2 | type = HR

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,2,0.46,0.89,1.28
2014,2,-1.67,-1.23,-0.17


# Rearranging Multi-Indices

In [51]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.800584
      2      0.413429
c     1      0.594400
      2      0.000588
b     1      0.785878
      2      0.822046
dtype: float64

In [52]:
#sort_index() & sortlevel()
data = data.sort_index()
data

char  int
a     1      0.800584
      2      0.413429
b     1      0.785878
      2      0.822046
c     1      0.594400
      2      0.000588
dtype: float64

# Stacking and unstacking indices
stacked multi-index -> simple two-dimensional representation

In [54]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [55]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [56]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [58]:
# used to recover the original series
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

# Index setting and resetting
index label -> to column

In [59]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [60]:
#Real world: column values -> to multindex
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

# Data Aggregations on Multi-Indices

In [62]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      1.82  0.72 -2.83 -0.14 -0.40 -0.48
     2      0.46  0.56  0.89 -0.27  1.28 -0.93
2014 1     -0.48 -0.58  0.54 -0.25  1.32 -1.38
     2     -1.67  0.79 -1.23  0.79 -0.17  0.75

In [63]:
data_mean = health_data.mean(level='year')
data_mean

subject    Bob         Guido           Sue       
type        HR   Temp     HR   Temp     HR   Temp
year                                             
2013     1.140  0.640 -0.970 -0.205  0.440 -0.705
2014    -1.075  0.105 -0.345  0.270  0.575 -0.315

In [65]:
#mean among columns
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,0.203333,-0.09
2014,-0.281667,0.02


# Aside: Panel Data

In [ ]:
Series      ->  one-dimensional
DataFrame     ->  2-dimensional
Panel      ->  3-dimensional
Panel4D     ->  4-dimensional